In [1]:
import torch

In [2]:
# Fonction pour extraire les données
import os
import gzip

def extract_gz_files(directory):
    """
    Extracts the contents of all .gz files in a directory and saves them in the same directory.
    
    Parameters:
    directory (str): Path to the directory containing the .gz files.
    """
    for filename in os.listdir(directory):
        if filename.endswith("nii.gz"):
            file_path = os.path.join(directory, filename)
            with gzip.open(file_path, "rb") as gz_file:
                uncompressed_data = gz_file.read()
            new_filename = filename[:-3]  # Remove the .gz extension
            new_file_path = os.path.join(directory, new_filename)
            with open(new_file_path, "wb") as new_file:
                new_file.write(uncompressed_data)
            os.remove(file_path)  # Delete the original .gz file


In [3]:
path1 = '/media/david/Elements/dev/DL/Monai-AI-Healthcare-Imaging/data/Task03_Liver/Task03_Liver/imagesTr'
path2 = '/media/david/Elements/dev/DL/Monai-AI-Healthcare-Imaging/data/Task03_Liver/Task03_Liver/imagesTs' 

In [4]:
os.listdir()

['.ipynb_checkpoints',
 'Computer-Vision-Healthcare',
 'data',
 'Preprocessing data.ipynb',
 'sources.txt']

In [5]:
paths = os.getcwd()
path_images_tr = os.path.join(paths, '', 'data/Task03_Liver/Task03_Liver/imagesTr')
path_dicom_file = os.path.join(paths, '', 'data/Task03_Liver/Task03_Liver/dicom_file')
path_dicom_groups = os.path.join(paths, '', 'data/Task03_Liver/Task03_Liver/dicom_groups')

In [6]:
# exemple
path_dicom_groups

'/media/david/Elements/dev/DL/Monai-AI-Healthcare-Imaging/data/Task03_Liver/Task03_Liver/dicom_groups'

## Labels

In [8]:
input_path_labels = os.path.join(path_dicom_file, '', 'labels')
input_path_labels

'/media/david/Elements/dev/DL/Monai-AI-Healthcare-Imaging/data/Task03_Liver/Task03_Liver/dicom_file/labels'

In [9]:
output_path_labels = os.path.join(path_dicom_groups, '', 'labels')
output_path_labels

'/media/david/Elements/dev/DL/Monai-AI-Healthcare-Imaging/data/Task03_Liver/Task03_Liver/dicom_groups/labels'

### Creation d'un groupe de 64 slices

On va convertir les 64 slices en 1 seul fichier nfti

In [11]:
from glob import glob
import shutil

In [12]:
for patient in glob(input_path_labels + '/*'):
    print(patient)

/media/david/Elements/dev/DL/Monai-AI-Healthcare-Imaging/data/Task03_Liver/Task03_Liver/dicom_file/labels/liver_00
/media/david/Elements/dev/DL/Monai-AI-Healthcare-Imaging/data/Task03_Liver/Task03_Liver/dicom_file/labels/liver_01
/media/david/Elements/dev/DL/Monai-AI-Healthcare-Imaging/data/Task03_Liver/Task03_Liver/dicom_file/labels/liver_02
/media/david/Elements/dev/DL/Monai-AI-Healthcare-Imaging/data/Task03_Liver/Task03_Liver/dicom_file/labels/liver_03
/media/david/Elements/dev/DL/Monai-AI-Healthcare-Imaging/data/Task03_Liver/Task03_Liver/dicom_file/labels/liver_04
/media/david/Elements/dev/DL/Monai-AI-Healthcare-Imaging/data/Task03_Liver/Task03_Liver/dicom_file/labels/liver_05
/media/david/Elements/dev/DL/Monai-AI-Healthcare-Imaging/data/Task03_Liver/Task03_Liver/dicom_file/labels/liver_06
/media/david/Elements/dev/DL/Monai-AI-Healthcare-Imaging/data/Task03_Liver/Task03_Liver/dicom_file/labels/liver_07
/media/david/Elements/dev/DL/Monai-AI-Healthcare-Imaging/data/Task03_Liver/Task0

In [35]:
patient_list = glob(input_path + '/*')
patient_list[0]

'/media/david/Elements/dev/DL/Monai-AI-Healthcare-Imaging/data/Task03_Liver/Task03_Liver/dicom_file/labels/liver_00'

* `os.path.basename()` : split au dernier '/' et récupère le nom du dossier ou du fichier 
* `os.path.normpath()` : supprime les sequences '.' et '..' inutiles dans le chemin  

In [49]:
groups_number = 64

for patient in glob(input_path_labels + '/*'):
    # patient_name = liver_00 (pour le 1er patient, ect...)
    patient_name = os.path.basename(os.path.normpath(patient))
    # glob(patient + '/*') crée une liste de tous les fichiers du liver_00 (pour le 1er patient)
    # on va récupérer le nombre total de fichier qu'on divise par le nombre de patients par groupe 
    # souhaités (dans notre cas, on veut 64 patients par groupe)
    # Attention : on ne prend que les groupes entier de 64 personnes
    # ex: si un groupe de patients est de 80 personnes, nous n'aurons qu'un seul groupe de 64 personnes
    # cad qu'on ne garde pas la partie non entière de 24 personnes (pour un problème d'accuracy en DL)
    number_folders = int(len(glob(patient + '/*')) / groups_number)
    
    # on va créer autant de sous-groupe de 64 patients que le nombre de number_folders
    for i in range(number_folders):
        # os.path.join(path, patient_name+'_' + str(i)) ==> path/patient_name_i
        output_path_name = os.path.join(output_path, patient_name + f'_{str(i)}')
        # os.mkdir(folder_name) : va créer pour chaque folder_name un folder à son nom
        os.mkdir(output_path_name)
        
        all_patient = glob(patient + '/*')
        # pour chaque patient dans la liste all_patient
        for i, file in enumerate(all_patient):
            # si le nombre de fichier de patient est égale à 64, on stop la loop
            if i == groups_number :
                break
            # shutil.move(file, new_path) : on bouge le file vers le new_path
            shutil.move(file, output_path_name)

On va créer une fonction qui va creer 64 slices pour les images

## Images

In [15]:
input_path_images = os.path.join(path_dicom_file, '', 'images')
input_path_images

'/media/david/Elements/dev/DL/Monai-AI-Healthcare-Imaging/data/Task03_Liver/Task03_Liver/dicom_file/images'

In [16]:
output_path_images = os.path.join(path_dicom_groups, '', 'images')
input_path_images

'/media/david/Elements/dev/DL/Monai-AI-Healthcare-Imaging/data/Task03_Liver/Task03_Liver/dicom_file/images'

In [50]:
def create_slices(input_path, output_path, nb_slices, source):
    input_path = os.path.join(input_path, '', source)
    output_path = os.path.join(output_path, '', source)
    
    for patient in glob(input_path + '/*'):
        patient_name = os.path.basename(os.path.normpath(patient))
        number_folders = int(len(glob(patient + '/*')) / nb_slices)
        
        for i in range(number_folders):
            output_path_name = os.path.join(output_path, patient_name + f'_{str(i)}')
            os.mkdir(output_path_name)
            all_patient = glob(patient + '/*')
            
            for i, file in enumerate(all_patient):
                if i == nb_slices :
                    break
                    
                shutil.move(file, output_path_name)

In [51]:
create_slices(input_path=path_dicom_file, output_path=path_dicom_groups, nb_slices=64, source='images')

## Convertir les fichiers dicom en nifties 

In [7]:
path_dicom_groups

'/media/david/Elements/dev/DL/Monai-AI-Healthcare-Imaging/data/Task03_Liver/Task03_Liver/dicom_groups'